# Technical Analysis Template: sRoC = WMA + RoC

Follow the trend the using smooth Rate of Change. The smooth function is WMA.

In [ ]:
import qnt.data as qndata
import qnt.stats as qnstats
import qnt.xr_talib as qnxrtalib
import qnt.graph as qngraph

# Data

In [ ]:
data = qndata.load_data(min_date="2014-01-01", forward_order=True, dims=("time", "field", "asset"))

# Calc output

In [ ]:
SROC_POSITIVE_TREND_LEVEL = 0.0125

wma = qnxrtalib.WMA(data.sel(field='close'), 290)
sroc = qnxrtalib.ROCP(wma, 35)

is_liquid = data.sel(field="is_liquid")
weights = is_liquid.where(sroc > SROC_POSITIVE_TREND_LEVEL)

weights = weights / weights.sum("asset", skipna=True)
output = weights.fillna(0.0)

## Stats and plots

In [ ]:
stat = qnstats.calc_stat(data, output)
display(stat.to_pandas().tail())

In [ ]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'equity'].values,  
    color="blue", 
    name="PnL (Equity)", 
    type="log"
)

In [ ]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'underwater'].values, 
    color="red", 
    name="Underwater Chart", 
    range_max=0
)

In [ ]:
SR_OFFSET = 252*3 + 290 + 35
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas()[SR_OFFSET:], 
    stat.loc[:, 'sharpe_ratio'][SR_OFFSET:], 
    color="purple", 
    name="Rolling SR"
)

In [ ]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'bias'].values, 
    color="gray", 
    name="Bias"
)

# Save output

In [ ]:
qndata.write_output(output)